<a href="https://colab.research.google.com/github/Asaf1h/university-database/blob/main/sql_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Faker

import sqlite3
from faker import Faker
import numpy as np
import os
from prettytable import PrettyTable

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def update_course_students_count(conn):
    # Connect to the SQLite database
    cursor = conn.cursor()

    # Check if the StudentsCount column exists in the Courses table
    cursor.execute("PRAGMA table_info(Courses)")
    columns = cursor.fetchall()
    students_count_exists = any(column[1] == 'StudentsCount' for column in columns)

    if students_count_exists:
        # Update the StudentsCount column with the sum of students for each course
        cursor.execute('UPDATE Courses SET StudentsCount = (SELECT COUNT(*) FROM Students WHERE Students.CourseID_1 = Courses.CourseID OR Students.CourseID_2 = Courses.CourseID OR Students.CourseID_3 = Courses.CourseID OR Students.CourseID_4 = Courses.CourseID)')
    else:
        # Add the StudentsCount column to the Courses table
        cursor.execute('ALTER TABLE Courses ADD COLUMN StudentsCount INTEGER DEFAULT 0')
        cursor.execute('UPDATE Courses SET StudentsCount = (SELECT COUNT(*) FROM Students WHERE Students.CourseID_1 = Courses.CourseID OR Students.CourseID_2 = Courses.CourseID OR Students.CourseID_3 = Courses.CourseID OR Students.CourseID_4 = Courses.CourseID)')

    # Commit the changes and close the connection
    conn.commit()



def Generate_DB(N):
    # Connect to the SQLite database
    conn = sqlite3.connect('university.db')
    cursor = conn.cursor()

    # Create the Contact table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Contact (
            ID INTEGER PRIMARY KEY,
            FirstName TEXT,
            LastName TEXT,
            Address TEXT,
            Email TEXT UNIQUE,
            Phone TEXT UNIQUE,
            Status TEXT,
            FieldOfStudy TEXT
        )
    ''')

    # Create the Students table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Students (
            ID INTEGER PRIMARY KEY,
            FirstName TEXT,
            LastName TEXT,
            GPA REAL CHECK (GPA >= 60 AND GPA <= 100),
            PaymentStatus TEXT,
            FieldOfStudy TEXT,
            CourseID_1 TEXT,
            CourseID_2 TEXT,
            CourseID_3  TEXT,
            CourseID_4  TEXT,
            FOREIGN KEY (ID) REFERENCES Contact(ID)
        )
    ''')

    # Create the lecturers table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS lecturers (
            ID INTEGER PRIMARY KEY,
            FirstName TEXT,
            LastName TEXT,
            FieldOfStudy TEXT,
            CourseID_1  TEXT,
            CourseID_2  TEXT,
            FOREIGN KEY (ID) REFERENCES Contact(ID)
        )
    ''')

    # Create the Courses table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Courses (
            CourseID INTEGER PRIMARY KEY,
            CourseName TEXT
        )
    ''')

    # Generate sample data using Faker
    fake = Faker()

    cursor.execute('SELECT Email FROM Contact')
    existing_phones = {row[0] for row in cursor.fetchall()}

     # Fetch existing emails from the database
    cursor.execute('SELECT Email FROM Contact')
    existing_emails = {row[0] for row in cursor.fetchall()}

    # Populate the Contact table with sample data
    for _ in range(N):
        first_name, last_name = fake.first_name(), fake.last_name()
        address = fake.address()

        email = generate_unique_email(existing_emails, fake)
        phone = generate_unique_phones(existing_phones, fake)
        status = fake.random_element(['worker', 'student', 'student', 'student',  'student', 'lecturer', 'lecturer'])
        field_of_study = fake.random_element(['Math', 'Computer Science', 'Psychology'])
        cursor.execute('INSERT INTO Contact (FirstName, LastName, Address, Email, Phone, Status, FieldOfStudy) VALUES (?, ?, ?, ?, ?, ?, ?)',
                      (first_name, last_name, address, email, phone, status, field_of_study))
    conn.commit()




    # Populate the Courses table with sample data
    cursor.execute('SELECT COUNT(*) FROM Contact WHERE Status = "lecturer"')
    num_of_Courses = cursor.fetchone()[0]*2

    for _ in range(num_of_Courses):
        course_name = fake.catch_phrase()
        cursor.execute('INSERT INTO Courses (CourseName) VALUES (?)', (course_name,))

    conn.commit()


    # Iterate over the Contact table and insert all the students into the Students table
    cursor.execute('SELECT ID, FirstName, LastName, FieldOfStudy FROM Contact WHERE Status IS "student"')
    students = cursor.fetchall()


    for student in students:
        contact_id, first_name, last_name, field_of_study = student

        # Fetch three random CourseIDs from the Courses table
        cursor.execute('SELECT CourseID FROM Courses ORDER BY RANDOM() LIMIT 4')
        course_ids = [row[0] for row in cursor.fetchall()]
        course_id_1, course_id_2, course_id_3, course_id_4 = course_ids

        mean, std_dev = 80, 10
        gpa = np.round(np.clip(np.random.normal(mean, std_dev), 60, 100), 3)
        payment_status = fake.random_element(elements=('Paid', 'Unpaid'))

        cursor.execute('INSERT INTO Students (ID, FirstName, LastName, GPA, PaymentStatus, FieldOfStudy, CourseID_1, CourseID_2, CourseID_3, CourseID_4) '
                        'VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', (contact_id, first_name, last_name, gpa, payment_status,
                                                            field_of_study, course_id_1, course_id_2, course_id_3, course_id_4))
        conn.commit()

    cursor.execute('SELECT ID, FirstName, LastName, FieldOfStudy FROM Contact WHERE Status IS "lecturer"')
    lecturers_data = cursor.fetchall()
    generate_lecturers(conn, lecturers_data)
    update_course_students_count(conn)
    conn.close()

def generate_lecturers(conn, lecturers_data):

    cursor = conn.cursor()
    # Get all the CourseIDs from the Courses table
    cursor.execute('SELECT CourseID FROM Courses')
    course_ids = cursor.fetchall()
    np.random.shuffle(course_ids)
    i = 0
    for lecturer in lecturers_data:
        contact_id, first_name, last_name, field_of_study = lecturer

        # Fetch three two CourseIDs from the Courses table
        cursor.execute('SELECT CourseID FROM Courses ORDER BY RANDOM() LIMIT 2')
        course_id_1, course_id_2 = course_ids[i][0], course_ids[i+1][0]
        i += 2
        cursor.execute('INSERT INTO lecturers (ID, FirstName, LastName, FieldOfStudy, CourseID_1, CourseID_2) '
                        'VALUES (?, ?, ?, ?, ?, ?)', (contact_id, first_name, last_name,
                                                            field_of_study, course_id_1, course_id_2))
    conn.commit()
    # Commit the changes and close the connection



In [ ]:
def delete_db():
  db_file = '/content/university.db'
  os.remove(db_file)

def get_table_data():
    conn = sqlite3.connect('university.db')
    cursor = conn.cursor()

    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = cursor.fetchall()

    data = {}
    for table in tables:
        table_name = table[0]
        cursor.execute(f"SELECT * FROM {table_name}")
        table_data = cursor.fetchall()

        # Create a PrettyTable instance and set the column names
        table_output = PrettyTable()
        cursor.execute(f"PRAGMA table_info({table_name})")
        columns = cursor.fetchall()
        column_names = [column[1] for column in columns]
        table_output.field_names = column_names

        # Add the data rows to the table
        for row in table_data:
            table_output.add_row(row)

        data[table_name] = table_output.get_string()

    conn.close()
    # Get and print the table data
    for table_name, table_data in data.items():
        print(f"Table: {table_name}")
        print(table_data)
        print()

def get_data_by_id(id):
    print('\n',' - *'*5)
    print(f'extract data for ID num: {id}:')
    conn = sqlite3.connect('university.db')
    cursor = conn.cursor()

    cursor.execute(f"SELECT * FROM Contact WHERE ID=?", (id,))
    contact_data = cursor.fetchall()

    cursor.execute(f"SELECT * FROM Students WHERE ID=?", (id,))
    student_data = cursor.fetchall()

    cursor.execute(f"SELECT * FROM lecturers WHERE ID=?", (id,))
    lecture_data = cursor.fetchall()

    conn.close()
    return contact_data, student_data, lecture_data


def get_data_by_name(name):
    print('\n',' - *'*5)
    print(f'extract data for the name: {name}:')
    conn = sqlite3.connect('university.db')
    cursor = conn.cursor()

    cursor.execute(f"SELECT * FROM Contact WHERE FirstName=? OR LastName=?", (name, name))
    contact_data = cursor.fetchall()

    cursor.execute("SELECT * FROM Students WHERE ID IN (SELECT ID FROM Contact WHERE FirstName = ? OR LastName = ?)", (name, name))
    student_data = cursor.fetchall()

    cursor.execute(f"SELECT * FROM lecturers WHERE ID=(SELECT ID FROM Contact WHERE FirstName=? OR LastName=?)", (name,name))

    lecture_data = cursor.fetchall()

    conn.close()
    return contact_data, student_data, lecture_data


def print_data(data):
    if any(data):
      for i, table in enumerate(data):
        print('\n',['Contact', 'Students','lecturers'][i], 'Table:')
        if table:
          print(table)
        else:
            print("No data found for the given name.")
    else:
      print('No data found for the given name')

def generate_unique_phones(existing_phones, fake):
    while True:
        phone = f'+972 {fake.msisdn()[5:]}'
        if phone not in existing_phones:
            return phone

def generate_unique_email(existing_emails, fake):
    while True:
        email = fake.email().rsplit('@', 1)[0] + '@ariel.ac.il'
        if email not in existing_emails:
            return email

def generate_people(name=None, N=1, status=None):
    conn = sqlite3.connect('university.db')

    cursor = conn.cursor()

    cursor.execute("SELECT MAX(ID) FROM Contact")
    last_id = cursor.fetchone()[0]

    cursor.execute('SELECT Email FROM Contact')
    existing_phones = {row[0] for row in cursor.fetchall()}

     # Fetch existing emails from the database
    cursor.execute('SELECT Email FROM Contact')
    existing_emails = {row[0] for row in cursor.fetchall()}

    for _ in range(N):
        if status == None:
           temp_status = fake.random_element(['worker', 'student', 'student', 'student',  'student', 'lecturer', 'lecturer'])
        else:
          temp_status = status
        last_id +=1
        first_name, last_name = fake.first_name(), fake.last_name()
        if name is not None:
            N = 1
            first_name = name
        address = fake.address()
        email =  generate_unique_email(existing_emails, fake)
        phone = generate_unique_phones(existing_phones, fake)
        field_of_study = fake.random_element(['Math', 'Computer Science', 'Psychology'])

        cursor.execute('INSERT INTO Contact (ID, FirstName, LastName, Address, Email, Phone, Status, FieldOfStudy) VALUES (?, ?, ?, ?, ?, ?, ?, ?)',
                  (last_id, first_name, last_name, address, email, phone, temp_status, field_of_study))
        conn.commit()


        if temp_status == 'student':
            cursor.execute('SELECT CourseID FROM Courses ORDER BY RANDOM() LIMIT 4')
            course_ids = [row[0] for row in cursor.fetchall()]
            course_ids += [None] * (4 - len(course_ids))
            course_id_1, course_id_2, course_id_3, course_id_4 = course_ids

            mean, std_dev = 80, 10
            gpa = np.round(np.clip(np.random.normal(mean, std_dev), 60, 100), 3)
            payment_status = fake.random_element(elements=('Paid', 'Unpaid'))

            cursor.execute('INSERT INTO Students (ID, FirstName, LastName, GPA, PaymentStatus, FieldOfStudy, CourseID_1, CourseID_2, CourseID_3, CourseID_4) '
                        'VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', (last_id, first_name, last_name, gpa, payment_status,
                                                            field_of_study, course_id_1, course_id_2, course_id_3, course_id_4))
        elif temp_status == 'lecturer':
              num_of_Courses =  2
              for _ in range(num_of_Courses):
                  course_name = fake.catch_phrase()
                  cursor.execute('INSERT INTO Courses (CourseName) VALUES (?)', (course_name,))

              lecturers_data = [(last_id, first_name, last_name, field_of_study)]
              generate_lecturers(conn, lecturers_data)

        conn.commit()
    print('\n', '- *'*5)
    if N == 1:
      print(f'{name} has been generated')
    else:
         print(f'{N} people have been generated')

    update_course_students_count(conn)
    conn.close()


#show off

In [ ]:
#delete_db()


In [ ]:
N = 10
Generate_DB(N)
get_table_data()

Table: Contact
+----+-------------+----------+---------------------------------+--------------------------+---------------+----------+------------------+
| ID |  FirstName  | LastName |             Address             |          Email           |     Phone     |  Status  |   FieldOfStudy   |
+----+-------------+----------+---------------------------------+--------------------------+---------------+----------+------------------+
| 1  |    Angela   |  Conway  |   10963 Wood Squares Apt. 344   |   nguzman@ariel.ac.il    | +972 39920268 | student  |    Psychology    |
|    |             |          |      Bradleyfort, FL 68064      |                          |               |          |                  |
| 2  |     Roy     |  Kelley  |         033 Patton Mall         |   rjackson@ariel.ac.il   | +972 89124640 | student  |    Psychology    |
|    |             |          |     Robinsonhaven, NH 42879     |                          |               |          |                  |
| 3  | Chris

In [ ]:
name = "Robert"
print_data(get_data_by_name(name))
print_data(get_data_by_id(5))
name = "Roy"
print_data(get_data_by_name(name))
generate_people(name=name,N=1)
print_data(get_data_by_name(name))
generate_people(N=20)
generate_people(N=60, status='student' )


  - * - * - * - * - *
extract data for the name: Robert:
No data found for the given name

  - * - * - * - * - *
extract data for ID num: 5:

 Contact Table:
[(5, 'Jessica', 'Morris', '18324 Marquez Harbors Suite 395\nMolinaburgh, VT 10553', 'perryjustin@ariel.ac.il', '+972 19029648', 'lecturer', 'Psychology')]

 Students Table:
No data found for the given name.

 lecturers Table:
[(5, 'Jessica', 'Morris', 'Psychology', '4', '2')]

  - * - * - * - * - *
extract data for the name: Roy:

 Contact Table:
[(2, 'Roy', 'Kelley', '033 Patton Mall\nRobinsonhaven, NH 42879', 'rjackson@ariel.ac.il', '+972 89124640', 'student', 'Psychology')]

 Students Table:
[(2, 'Roy', 'Kelley', 70.081, 'Unpaid', 'Psychology', '1', '2', '3', '4')]

 lecturers Table:
No data found for the given name.

 - *- *- *- *- *
Roy has been generated

  - * - * - * - * - *
extract data for the name: Roy:

 Contact Table:
[(2, 'Roy', 'Kelley', '033 Patton Mall\nRobinsonhaven, NH 42879', 'rjackson@ariel.ac.il', '+972 8912

In [ ]:
get_table_data()

Table: Contact
+----+-------------+------------+-------------------------------------+------------------------------+---------------+----------+------------------+
| ID |  FirstName  |  LastName  |               Address               |            Email             |     Phone     |  Status  |   FieldOfStudy   |
+----+-------------+------------+-------------------------------------+------------------------------+---------------+----------+------------------+
| 1  |    Angela   |   Conway   |     10963 Wood Squares Apt. 344     |     nguzman@ariel.ac.il      | +972 39920268 | student  |    Psychology    |
|    |             |            |        Bradleyfort, FL 68064        |                              |               |          |                  |
| 2  |     Roy     |   Kelley   |           033 Patton Mall           |     rjackson@ariel.ac.il     | +972 89124640 | student  |    Psychology    |
|    |             |            |       Robinsonhaven, NH 42879       |                    